# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df=pd.read_csv("../WeatherPy/output/cities.csv")
cities_df.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Salamanca,40.8333,-6.0000,56.98,40,0,4.85,ES,1628468694
1,Jamestown,42.0970,-79.2353,78.39,82,1,3.00,US,1628468794
2,Richards bay,-28.7830,32.0377,64.09,57,1,10.16,ZA,1628468866
3,Hualmay,-11.0964,-77.6139,62.56,83,40,14.09,PE,1628468866
4,Punta arenas,-53.1500,-70.9167,41.11,75,90,12.66,CL,1628468866


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations= cities_df[["Lat","Long"]].astype(float)

#Convert humidity to float and store
humidity=cities_df["Humidity"].astype(float)

In [5]:
#Create a heamap layer for humidity

#Set center point and zoom level first
center_coords=(32.74, -16.93) #to center the map
fig=gmaps.figure(center=center_coords, zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations, weights=humidity,
                              dissipating=False, max_intensity=100,
                              point_radius=2)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# ideal weather condition is between 70 and 90 degrees, windspeed less than 15mph,
# humidity less than 40% and cloudiness less than 5%.

ideal_weather_df=cities_df[(cities_df["Max Temp"]>65) &
                         (cities_df["Max Temp"]<90) &
                         (cities_df["Wind Speed"]<15) &
                         (cities_df["Humidity"]<30) &
                         (cities_df["Cloudiness"]<25)]
ideal_weather_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
122,Gazli,40.1333,63.4500,83.32,21,0,10.65,UZ,1628468900
178,Montrose,38.4166,-108.2673,76.73,14,0,10.16,US,1628468918
187,Cheyenne,41.1400,-104.8203,87.24,18,1,9.22,US,1628468921
255,Mirabad,30.4362,61.8383,87.80,15,0,14.45,AF,1628468884
258,Gazanjyk,39.2446,55.5154,87.42,17,0,13.29,TM,1628468943
276,Sakakah,29.9697,40.2064,89.17,21,0,8.84,SA,1628468950
300,Riyadh,24.6877,46.7219,89.74,22,23,2.82,SA,1628468957
349,Cedar city,37.6775,-113.0619,88.39,8,1,14.97,US,1628468755
421,Tagab,34.8550,69.6492,73.31,25,0,5.39,AF,1628468992
446,Adrar,20.5022,-10.0711,86.94,20,11,10.56,MR,1628469000


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# create new column to hold Hotel Name

ideal_weather_df['Hotel Name']=''

# reposition columns

ideal_weather_df=ideal_weather_df[["City", "Country", "Hotel Name", "Lat", "Long", "Max Temp", "Humidity", "Cloudiness",
                                 "Wind Speed", "Date"]]

ideal_weather_df

C:\Users\Samantha\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,Hotel Name,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
122,Gazli,UZ,,40.1333,63.4500,83.32,21,0,10.65,1628468900
178,Montrose,US,,38.4166,-108.2673,76.73,14,0,10.16,1628468918
187,Cheyenne,US,,41.1400,-104.8203,87.24,18,1,9.22,1628468921
255,Mirabad,AF,,30.4362,61.8383,87.80,15,0,14.45,1628468884
258,Gazanjyk,TM,,39.2446,55.5154,87.42,17,0,13.29,1628468943
276,Sakakah,SA,,29.9697,40.2064,89.17,21,0,8.84,1628468950
300,Riyadh,SA,,24.6877,46.7219,89.74,22,23,2.82,1628468957
349,Cedar city,US,,37.6775,-113.0619,88.39,8,1,14.97,1628468755
421,Tagab,AF,,34.8550,69.6492,73.31,25,0,5.39,1628468992
446,Adrar,MR,,20.5022,-10.0711,86.94,20,11,10.56,1628469000


In [8]:
# find a closest hotel of each type to coordinates

#params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key,
}

# use the lat/long to identify hotels
for index,row in ideal_weather_df.iterrows():
    #get lat,lng from df
    lat=row["Lat"]
    lng=row["Long"]
    city=row["City"]
    
    #change location each iteration while leaving original params in place
    params["location"]=f"{lat},{lng}"
    
    #Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #make request and print url
    name_address=requests.get(base_url, params=params)
    
#     print(name_address.url)
    
    #convert to jason
    name_address=name_address.json()
    
    try:
        ideal_weather_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except IndexError:
        ideal_weather_df.loc[index, "Hotel Name"] = f"**No hotel nearby**"
        print(f"No hotel nearby for {city}")
        

ideal_weather_df

No hotel nearby for Gazli
No hotel nearby for Montrose
No hotel nearby for Mirabad
No hotel nearby for Tagab
No hotel nearby for Adrar
No hotel nearby for Awbari


,City,Country,Hotel Name,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
122,Gazli,UZ,**No hotel nearby**,40.1333,63.4500,83.32,21,0,10.65,1628468900
178,Montrose,US,**No hotel nearby**,38.4166,-108.2673,76.73,14,0,10.16,1628468918
187,Cheyenne,US,Red Lion Hotel & Conference Center Cheyenne,41.1400,-104.8203,87.24,18,1,9.22,1628468921
255,Mirabad,AF,**No hotel nearby**,30.4362,61.8383,87.80,15,0,14.45,1628468884
258,Gazanjyk,TM,Railway motel,39.2446,55.5154,87.42,17,0,13.29,1628468943
276,Sakakah,SA,Raoum Inn Hotel,29.9697,40.2064,89.17,21,0,8.84,1628468950
300,Riyadh,SA,Four Seasons Hotel,24.6877,46.7219,89.74,22,23,2.82,1628468957
349,Cedar city,US,Best Western Plus Cedar City,37.6775,-113.0619,88.39,8,1,14.97,1628468755
421,Tagab,AF,**No hotel nearby**,34.8550,69.6492,73.31,25,0,5.39,1628468992
446,Adrar,MR,**No hotel nearby**,20.5022,-10.0711,86.94,20,11,10.56,1628469000


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
locations = ideal_weather_df[["Lat", "Long"]]

In [11]:
# Add marker layer ontop of heat map
fig=gmaps.figure(center=center_coords, zoom_level=1.5)
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Save the figure
plt.savefig("output/Final_Heatmap.png")

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>